<a href="https://colab.research.google.com/github/rjac-ml/Gender-Bias-Wiki/blob/main/notebooks/SentenceEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!rm -r sample_data/
!pip install datasets transformers sentence_transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
import os 
ram_gb = virtual_memory().total / 1e9
NCPU = os.cpu_count()

print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Number  of CPU {}\n".format(NCPU))

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 54.8 gigabytes of available RAM

Number  of CPU 8

Fri May 20 22:27:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                    

In [4]:
%%capture
!tar -xvzf /content/drive/MyDrive/eautoclub/similarity_jsonl.tar.gz 

In [5]:
!cp -r /content/drive/MyDrive/eautoclub/model /content/

In [25]:
import pandas as pd
import numpy as np
from datasets import load_from_disk, load_dataset
import math
import torch
from transformers import (
    AutoModelForMaskedLM
    ,pipeline
    ,AutoTokenizer
    ,DataCollatorForLanguageModeling
    ,DataCollatorForWholeWordMask
    ,Trainer
    ,TrainingArguments
)
from sentence_transformers import SentenceTransformer, models, InputExample,losses
from torch.utils.data import DataLoader
from torch import nn

# Data Step

In [8]:
data_source = "similarity"
files = os.listdir(data_source)
abs_files = ["{}/{}".format(data_source,f) for f in files if ".jsonl" in f]
df = load_dataset("json",data_files=abs_files)
df = df.get("train")

Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Using custom data configuration default-3a4d8ba0a605c96f


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3a4d8ba0a605c96f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
df

Dataset({
    features: ['sentence', 'target', 'similarity'],
    num_rows: 128386
})

In [10]:
df = df.to_pandas()
train_examples = df.apply(lambda line: InputExample(texts=[line["sentence"],line["target"]], label=line["similarity"]*1.00),axis=1).values
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=250)

# Modeling Step

In [ ]:
word_embedding_model = models.Transformer('./model', max_seq_length=64)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=300, activation_function=nn.Tanh())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_loss = losses.CosineSimilarityLoss(model)

In [17]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4, warmup_steps=100)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/514 [00:00<?, ?it/s]

In [29]:
model.save("./sentence_encoder")